<a href="https://colab.research.google.com/github/Randon-Myntra-HackerRamp-21/Acne-type/blob/main/Acne_types_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import splitfolders
import tensorflow_hub as hub
from tensorflow.keras import layers
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
splitfolders.ratio(r'.\acne_images', output="output", seed=1337, ratio=(.8, 0.2)) 

In [3]:
def filter_hidden_directories(directory):
    return [subdir for subdir in os.listdir(directory) if not subdir.startswith('.')]



In [4]:
# Preprocess data (get all of the pixel values between 0 & 1, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [5]:

# Setup paths to our data directories
train_dir = 'output/train'
test_dir = 'output/val'

# Filter hidden directories
filtered_train_dirs = filter_hidden_directories(train_dir)
filtered_test_dirs = filter_hidden_directories(test_dir)

In [6]:

# Import data from filtered directories and turn it into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=(224, 224),
                                               classes=filtered_train_dirs,
                                               class_mode="categorical",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               target_size=(224, 224),
                                               classes=filtered_test_dirs,
                                               class_mode="categorical",
                                               seed=42)

Found 883 images belonging to 4 classes.
Found 223 images belonging to 4 classes.


In [7]:
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

def create_model(model_url, num_classes=3):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

  Args:
    model_url(str): A TensorFlow Hub feature extraction URL.
    num_classes(int): Number of output neurons in the output layer, 
      should be equal to number of target classes, default = 10

  Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor 
    layer and Dense output layer with num_classes output neurons.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable = False,   # freeze the already learned patterns
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3,))  # define the input image shape
  # Create our image model
  model = tf.keras.Sequential([
     feature_extractor_layer,    # use the feature extraction layer as the base
     layers.Dense(num_classes, activation="softmax", name="output_layer")   # create our own output layer
  ])
  return model

In [8]:
# Create EfficientNet model
efficientnet_model = create_model(efficientnet_url,
                            num_classes=4)



In [9]:
# Compile
efficientnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
# Fit the model
efficientnet_history = efficientnet_model.fit(train_data,
                                  epochs=5,
                                  validation_data=valid_data)   # name of log files

Epoch 1/5
28/28 [==============================] - 144s 5s/step - loss: 1.1557 - accuracy: 0.4507 - val_loss: 1.0535 - val_accuracy: 0.4709
Epoch 2/5
28/28 [==============================] - 118s 4s/step - loss: 0.9808 - accuracy: 0.5515 - val_loss: 0.9712 - val_accuracy: 0.5202
Epoch 3/5
18/28 [==================>...........] - ETA: 34s - loss: 0.9116 - accuracy: 0.5990

In [ ]:
efficientnet_model.summary()

In [ ]:
efficientnet_model.save('saved_model/acne_model')

In [ ]:
model = tf.keras.models.load_model('saved_model/acne_model')

# Check its architecture
model.summary()

In [ ]:
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [ ]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into (224,224,3).
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
# Function to make a prediction and plot the result
def pred_and_plot(model, filename, class_names):
    img = load_and_prep_image(filename)
    pred = model.predict(tf.expand_dims(img, axis=0))

    if len(pred[0]) > 1:
        pred_class = class_names[tf.argmax(pred[0])]
    else:
        pred_class = class_names[int(tf.round(pred[0]))]
    
    print('Prediction Probabilities:', pred[0])

    plt.imshow(img)
    plt.title(f"Prediction: {pred_class}")
    plt.axis(False);

In [ ]:
# Make a prediction using the loaded model
pred_and_plot(model=model, 
              filename=r"C:\Users\imana\CW AI\Skyn\ML\Skin_metrics\Acne\test_image.jpeg", 
              class_names=class_names)